Import

In [2]:
import json
import numpy as np
import pandas as pd
import seaborn as sb

data = json.load(open('data/realdonaldtrump.json', 'r'))

Prepare Variables

In [7]:
tweets = []
for key, value in data.items():
    value['timestamp'] = key
    tweets.append(value.__setitem__('timestamp', key))

tweets

Happy New Year!', 'retweet_count': 30658, 'favorite_count': 109247, 'timestamp': '1212121026072592384'}
{'created_at': 'Tue Sep 27 01:32:25 +0000 2016', 'full_text': 'A Clinton economy = more taxes and more spending! #DebateNight https://t.co/oFlaAhrwe5', 'retweet_count': 6547, 'favorite_count': 14017, 'timestamp': '780580825234415616'}
{'created_at': 'Mon Nov 11 02:25:43 +0000 2019', 'full_text': 'A great new book just out, “The Plot Against the President. The True Story Of How Congressman Devin Nunes Uncovered the Biggest Political Scandal In U.S. History.” Shows very bad and corrupt people on the other side. Check it out! https://t.co/KQETvKO6Q9', 'retweet_count': 10819, 'favorite_count': 35818, 'timestamp': '1193716405771931650'}
{'created_at': 'Tue May 19 22:02:10 +0000 2015', 'full_text': 'With Sen. Elizabeth Dole &amp; @DoleFoundation Caregiver Fellows. Tremendous people caring for our military &amp; veterans! http://t.co/zGr7jLhj5B', 'retweet_count': 55, 'favorite_count': 91, '

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,